# CEIA - Procesamiento de Lenguaje Natural

## Estudiante: a2124 - Ricardo Silvera

--- 

# Desafío 3

### Consigna
- Seleccionar un corpus de texto sobre el cual entrenar el modelo de lenguaje.
- Realizar el pre-procesamiento adecuado para tokenizar el corpus, estructurar el dataset y separar entre datos de entrenamiento y validación.
- Proponer arquitecturas de redes neuronales basadas en unidades recurrentes para implementar un modelo de lenguaje.
- Con el o los modelos que consideren adecuados, generar nuevas secuencias a partir de secuencias de contexto con las estrategias de greedy search y beam search determístico y estocástico. En este último caso observar el efecto de la temperatura en la generación de secuencias.


### Sugerencias
- Durante el entrenamiento, guiarse por el descenso de la perplejidad en los datos de validación para finalizar el entrenamiento. Para ello se provee un callback.
- Explorar utilizar SimpleRNN (celda de Elman), LSTM y GRU.
- rmsprop es el optimizador recomendado para la buena convergencia. No obstante se pueden explorar otros.


---


In [1]:
import tensorflow as tf
import os

print("="*70)
print("CONFIGURACIÓN DE GPU OPTIMIZADA")
print("="*70)

print(f"\nTensorFlow version: {tf.__version__}")
print(f"¿Construido con CUDA?: {tf.test.is_built_with_cuda()}")

# Listar dispositivos físicos
print("\n--- Dispositivos disponibles ---")
gpus = tf.config.list_physical_devices('GPU')
cpus = tf.config.list_physical_devices('CPU')
print(f"CPUs: {len(cpus)}")
print(f"GPUs: {len(gpus)}")

if gpus:
    print(f"\n✓ GPU detectada: {gpus[0].name}")
    
    try:
        # Configurar crecimiento de memoria (CRÍTICO para GTX 1050)
        for gpu in gpus:
            tf.config.experimental.set_memory_growth(gpu, True)
        
        # Limitar memoria a 3GB
        tf.config.set_logical_device_configuration(
            gpus[0],
            [tf.config.LogicalDeviceConfiguration(memory_limit=3072)])
        
        print("✓ Crecimiento de memoria GPU: ACTIVADO")
        print("✓ Límite de memoria GPU: 3GB")
        
        # Configurar para forzar operaciones en GPU
        tf.config.set_soft_device_placement(False)
        
        # Test simple en GPU
        with tf.device('/GPU:0'):
            a = tf.constant([[1.0, 2.0], [3.0, 4.0]])
            b = tf.constant([[1.0, 1.0], [0.0, 1.0]])
            c = tf.matmul(a, b)
        
        print(f"✓ Test de operación en GPU: EXITOSO")
        print(f"  Dispositivo: {c.device}")
        print("✓ Soft device placement: DESACTIVADO (forzar GPU)")
        
    except RuntimeError as e:
        print(f"\n⚠ Error configurando GPU: {e}")
else:
    print("\n⚠ No se detectaron GPUs")

print("\n" + "="*70)

2025-12-07 11:41:14.150716: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:479] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2025-12-07 11:41:14.175513: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:10575] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2025-12-07 11:41:14.175550: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1442] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2025-12-07 11:41:14.191894: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2025-12-07 11:41:15.168560: W tensorflow/compiler/tf

CONFIGURACIÓN DE GPU OPTIMIZADA

TensorFlow version: 2.16.2
¿Construido con CUDA?: True

--- Dispositivos disponibles ---
CPUs: 1
GPUs: 1

✓ GPU detectada: /physical_device:GPU:0
✓ Crecimiento de memoria GPU: ACTIVADO
✓ Límite de memoria GPU: 3GB
✓ Test de operación en GPU: EXITOSO
  Dispositivo: /job:localhost/replica:0/task:0/device:GPU:0
✓ Soft device placement: DESACTIVADO (forzar GPU)



2025-12-07 11:41:16.924601: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:998] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2025-12-07 11:41:16.968976: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:998] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2025-12-07 11:41:16.969282: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:998] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-

In [ ]:
import tensorflow as tf
import os

print("="*70)
print("CONFIGURACIÓN DE GPU")
print("="*70)

print(f"\nTensorFlow version: {tf.__version__}")
print(f"¿Construido con CUDA?: {tf.test.is_built_with_cuda()}")

# Listar dispositivos físicos
print("\n--- Dispositivos disponibles ---")
gpus = tf.config.list_physical_devices('GPU')
cpus = tf.config.list_physical_devices('CPU')
print(f"CPUs: {len(cpus)}")
print(f"GPUs: {len(gpus)}")

if gpus:
    print(f"\n✓ GPU detectada: {gpus[0].name}")
    
    try:
        # Configurar crecimiento de memoria (CRÍTICO para GTX 1050)
        for gpu in gpus:
            tf.config.experimental.set_memory_growth(gpu, True)
        
        # Limitar memoria a 2.5GB (dejar 1.5GB libres para el sistema)
        tf.config.set_logical_device_configuration(
            gpus[0],
            [tf.config.LogicalDeviceConfiguration(memory_limit=2560)])
        
        print("✓ Crecimiento de memoria GPU: ACTIVADO")
        print("✓ Límite de memoria GPU: 2.5GB (dejando margen para el sistema)")
        
        # Test simple en GPU
        with tf.device('/GPU:0'):
            a = tf.constant([[1.0, 2.0], [3.0, 4.0]])
            b = tf.constant([[1.0, 1.0], [0.0, 1.0]])
            c = tf.matmul(a, b)
        
        print(f"✓ Test de operación en GPU: EXITOSO")
        print(f"  Dispositivo: {c.device}")
        
        # Información de GPUs lógicas
        logical_gpus = tf.config.list_logical_devices('GPU')
        print(f"\nGPUs lógicas configuradas: {len(logical_gpus)}")
        
    except RuntimeError as e:
        print(f"\n⚠ Error configurando GPU: {e}")
else:
    print("\n⚠ No se detectaron GPUs")
    print("El entrenamiento se realizará en CPU (será más lento)")
    print("\nPara habilitar GPU, ejecuta en terminal:")
    print("  pip install tensorflow[and-cuda]==2.16.2")

print("\n" + "="*70)

## Imporación de librerias 

In [2]:
import random
import io
import pickle

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split

from tensorflow import keras
from tensorflow.keras import layers
from keras.utils import to_categorical
from keras.models import Sequential
from keras.layers import Dense, LSTM, Embedding, Dropout
from tensorflow.keras.losses import SparseCategoricalCrossentropy
from tensorflow.keras.utils import pad_sequences
from keras.layers import Input, TimeDistributed, CategoryEncoding, SimpleRNN, Dense, GRU
from keras.models import Model, Sequential

## Selección del corpus

Seleccione como corpus el texto de la novela Crimen y Castigo de Fiódor Mijáilovich Dostoyevski, extraído del sitio web https://www.textos.info/. El texto se encuentra en español y me resulta familiar para analizar contextos o referencias que puedan resultar de la generación de secuencias que debe realizar el modelo a entrenar, lo que facilitará el análisis de los resultados, y por otro lado es agradable trabajar con material literario que disfruté en algún momento. 

In [3]:
# Lectura del corpus
with open('crimen-ycastigo.txt', 'r', encoding='utf-8') as f:
    article_text = f.read()

# Pasar todo el texto a minúscula
article_text = article_text.lower()

# Verificar la longitud del corpus
print(f"Longitud del corpus: {len(article_text)} caracteres")
print(f"Primeros 500 caracteres:\n{article_text[:500]}")

Longitud del corpus: 1044667 caracteres
Primeros 500 caracteres:
primera parte
i
una tarde muy calurosa de principios de julio, salió del cuartito que ocupaba, junto al techo de una gran casa de cinco pisos, un joven, que, lentamente y con aire irresoluto, se dirigió hacia el puente de k***.

tuvo suerte, al bajar la escalera, de no encontrarse a su patrona que habitaba en el piso cuarto, y cuya cocina, que tenía la puerta constantemente sin cerrar, daba a la escalera. cuando salía el joven, había de pasar forzosamente bajo el fuego del enemigo, y cada vez qu


## Pre-procesamiento

### Construcción del vocabulario

In [4]:
# El vocabulario es el conjunto único de caracteres que existe en todo el texto
chars_vocab = set(article_text)

# La longitud del vocabulario de caracteres es:
vocab_size = len(chars_vocab)
print(f"Tamaño del vocabulario: {vocab_size} caracteres únicos")
print(f"Caracteres: {sorted(chars_vocab)}")

Tamaño del vocabulario: 62 caracteres únicos
Caracteres: ['\n', ' ', '!', '(', ')', '*', ',', '-', '.', '0', '1', '2', '3', '4', '5', '7', '9', ':', ';', '?', 'a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j', 'k', 'l', 'm', 'n', 'o', 'p', 'q', 'r', 's', 't', 'u', 'v', 'w', 'x', 'y', 'z', '¡', '«', '»', '¿', 'á', 'ä', 'é', 'í', 'ï', 'ñ', 'ó', 'ö', 'ú', 'û', 'ü', '—']


In [5]:
# Creo los diccionarios que asignan índices a caracteres y viceversa
# El diccionario char2idx servirá como tokenizador
char2idx = {k: v for v, k in enumerate(chars_vocab)}
idx2char = {v: k for k, v in char2idx.items()}

Visualización de los diccionarios

In [6]:
print("Char to Index:")
print(char2idx)
print("Index to Char:")
print(idx2char)

Char to Index:
{'d': 0, ')': 1, 'á': 2, 'û': 3, 'ñ': 4, '1': 5, 'y': 6, 'f': 7, 'q': 8, 'ü': 9, '2': 10, ':': 11, '—': 12, '!': 13, '3': 14, '?': 15, 'i': 16, 'ä': 17, 'g': 18, 'k': 19, 'w': 20, 'x': 21, 'n': 22, 'j': 23, 'h': 24, '(': 25, 'ö': 26, 'l': 27, 'p': 28, '4': 29, 'u': 30, 'ó': 31, 'ú': 32, 't': 33, 'z': 34, 'o': 35, '¡': 36, '7': 37, 'e': 38, '9': 39, ',': 40, ';': 41, '*': 42, '\n': 43, '0': 44, '«': 45, '.': 46, 'b': 47, '5': 48, 'ï': 49, 'a': 50, 'í': 51, 's': 52, '¿': 53, ' ': 54, 'é': 55, '-': 56, 'c': 57, 'v': 58, '»': 59, 'm': 60, 'r': 61}
Index to Char:
{0: 'd', 1: ')', 2: 'á', 3: 'û', 4: 'ñ', 5: '1', 6: 'y', 7: 'f', 8: 'q', 9: 'ü', 10: '2', 11: ':', 12: '—', 13: '!', 14: '3', 15: '?', 16: 'i', 17: 'ä', 18: 'g', 19: 'k', 20: 'w', 21: 'x', 22: 'n', 23: 'j', 24: 'h', 25: '(', 26: 'ö', 27: 'l', 28: 'p', 29: '4', 30: 'u', 31: 'ó', 32: 'ú', 33: 't', 34: 'z', 35: 'o', 36: '¡', 37: '7', 38: 'e', 39: '9', 40: ',', 41: ';', 42: '*', 43: '\n', 44: '0', 45: '«', 46: '.', 47: '

### Tokenización

In [7]:
tokenized_text = [char2idx[ch] for ch in article_text]
print(f"Longitud del texto tokenizado: {len(tokenized_text)}")
print(f"Primeros 100 tokens: {tokenized_text[:100]}")

Longitud del texto tokenizado: 1044667
Primeros 100 tokens: [28, 61, 16, 60, 38, 61, 50, 54, 28, 50, 61, 33, 38, 43, 16, 43, 30, 22, 50, 54, 33, 50, 61, 0, 38, 54, 60, 30, 6, 54, 57, 50, 27, 30, 61, 35, 52, 50, 54, 0, 38, 54, 28, 61, 16, 22, 57, 16, 28, 16, 35, 52, 54, 0, 38, 54, 23, 30, 27, 16, 35, 40, 54, 52, 50, 27, 16, 31, 54, 0, 38, 27, 54, 57, 30, 50, 61, 33, 16, 33, 35, 54, 8, 30, 38, 54, 35, 57, 30, 28, 50, 47, 50, 40, 54, 23, 30, 22, 33, 35]


### Selección del tamaño de contexto

Como la tokenización es por caracter, la cantodad de tokens es igual a la cantidad de caracteres en el texto: 1044667. Todo el corpus puede ser considerado un documento  y el tamaño de contexto puede ser elegido con mayor libertad en comparación a un modelo tokenizado por palabras.

In [8]:
# Seleccionamos el tamaño de contexto
max_context_size = 100

### Estructuración del dataset

Separo el dataset en entrenamiento y validación. Utilizo una 10% para validación.

In [9]:
#p_val es la proporcion del corpus que uso para validación 
p_val = 0.1

# num_val es la cantidad de secuencias de tamaño max_context_size que se usará en validación
num_val = int(np.ceil(len(tokenized_text) * p_val / max_context_size))

print(f"Número de secuencias para validación: {num_val}")

Número de secuencias para validación: 1045


In [10]:
# Datos de entrenamiento
train_text = tokenized_text[:-num_val * max_context_size]
# Datos de validación
val_text = tokenized_text[-num_val * max_context_size:]

print(f"Longitud del texto de entrenamiento: {len(train_text)}")
print(f"Longitud del texto de validación: {len(val_text)}")

Longitud del texto de entrenamiento: 940167
Longitud del texto de validación: 104500


Creación de las secuencias

In [11]:
# Secuencias de validación
tokenized_sentences_val = [val_text[init * max_context_size:init * (max_context_size + 1)] 
                           for init in range(num_val)]

# Secuencias de entrenamiento
tokenized_sentences_train = [train_text[init:init + max_context_size] 
                             for init in range(len(train_text) - max_context_size + 1)]

print(f"Número de secuencias de entrenamiento: {len(tokenized_sentences_train)}")
print(f"Número de secuencias de validación: {len(tokenized_sentences_val)}")

Número de secuencias de entrenamiento: 940068
Número de secuencias de validación: 1045


In [12]:
# Estructuramos el problema como many-to-many:
# Entrada: secuencia de tokens [x_0, x_1, ..., x_N]
# Target: secuencia de tokens [x_1, x_2, ..., x_{N+1}]

X = np.array(tokenized_sentences_train[:-1])
y = np.array(tokenized_sentences_train[1:])

print(f"Shape de X: {X.shape}")
print(f"Shape de y: {y.shape}")
print(f"\nEjemplo - Primeros 10 tokens de X[0]: {X[0, :10]}")
print(f"Ejemplo - Primeros 10 tokens de y[0]: {y[0, :10]}")

Shape de X: (940067, 100)
Shape de y: (940067, 100)

Ejemplo - Primeros 10 tokens de X[0]: [28 61 16 60 38 61 50 54 28 50]
Ejemplo - Primeros 10 tokens de y[0]: [61 16 60 38 61 50 54 28 50 61]


## Arquitecturas

### Callback de Perplejidad

Implemento el callback ad-hoc para calcular la perplejidad al final de cada epoch de entrenamiento sobre un conjunto de datos de validación. La perplejidad es una métrica cuantitativa para evaluar la calidad de la generación de secuencias. Además implementa Early Stopping si la perplejidad no mejora después de `patience` epochs.

In [13]:
class PplCallback(keras.callbacks.Callback):
    '''
    Este callback es una solución ad-hoc para calcular al final de cada epoch de
    entrenamiento la métrica de Perplejidad sobre un conjunto de datos de validación.
    La perplejidad es una métrica cuantitativa para evaluar la calidad de la generación de secuencias.
    Además implementa la finalización del entrenamiento (Early Stopping)
    si la perplejidad no mejora después de `patience` epochs.
    '''

    def __init__(self, val_data, history_ppl, patience=5, batch_size=64):
        # El callback lo inicializamos con secuencias de validación sobre las cuales
        # mediremos la perplejidad
        self.val_data = val_data
        self.batch_size = batch_size  # Batch size para predicciones (menor para evitar OOM)

        self.target = []
        self.padded = []

        count = 0
        self.info = []
        self.min_score = np.inf
        self.patience_counter = 0
        self.patience = patience

        # nos movemos en todas las secuencias de los datos de validación
        for seq in self.val_data:
            len_seq = len(seq)
            # armamos todas las subsecuencias
            subseq = [seq[:i] for i in range(1, len_seq)]
            self.target.extend([seq[i] for i in range(1, len_seq)])

            if len(subseq) != 0:
                self.padded.append(pad_sequences(subseq, maxlen=max_context_size, padding='pre'))
                self.info.append((count, count + len_seq))
                count += len_seq

        self.padded = np.vstack(self.padded)

    def on_epoch_end(self, epoch, logs=None):
        # en `scores` iremos guardando la perplejidad de cada secuencia
        scores = []

        # Predicciones en batches pequeños para evitar OOM
        predictions = []
        num_samples = len(self.padded)
        
        for i in range(0, num_samples, self.batch_size):
            batch = self.padded[i:i + self.batch_size]
            batch_pred = self.model.predict(batch, verbose=0)
            predictions.append(batch_pred)
        
        # Concatenar todas las predicciones
        predictions = np.vstack(predictions)

        # para cada secuencia de validación
        for start, end in self.info:
            # en `probs` iremos guardando las probabilidades de los términos target
            probs = [predictions[idx_seq, -1, idx_vocab] 
                    for idx_seq, idx_vocab in zip(range(start, end), self.target[start:end])]

            # calculamos la perplejidad por medio de logaritmos
            scores.append(np.exp(-np.sum(np.log(probs + 1E-10)) / (end - start)))

        # promediamos todos los scores e imprimimos el valor promedio
        current_score = np.mean(scores)
        history_ppl.append(current_score)
        print(f'\nmean perplexity: {current_score}\n')

        # chequeamos si tenemos que detener el entrenamiento
        if current_score < self.min_score:
            self.min_score = current_score
            self.model.save("best_model.keras")
            print("Saved new model!")
            self.patience_counter = 0
        else:
            self.patience_counter += 1
            if self.patience_counter == self.patience:
                print("Stopping training...")
                self.model.stop_training = True

In [14]:
# Función para crear diferentes arquitecturas de modelos
def crear_modelo(tipo_rnn='SimpleRNN', hidden_units=200, dropout=0.1, recurrent_dropout=0.1):
    """
    Crea un modelo de lenguaje con la arquitectura especificada.
    
    Args:
        tipo_rnn: tipo de capa recurrente ('SimpleRNN', 'LSTM', 'GRU')
        hidden_units: número de unidades ocultas en la capa recurrente
        dropout: tasa de dropout
        recurrent_dropout: tasa de dropout recurrente
    
    Returns:
        modelo compilado
    """
    with tf.device('/GPU:0' if tf.config.list_physical_devices('GPU') else '/CPU:0'):
        
        model = Sequential()
        
        # Capa de codificación one-hot
        model.add(TimeDistributed(CategoryEncoding(num_tokens=vocab_size, output_mode="one_hot"),
                                input_shape=(None, 1)))
        
        # Capa recurrente según el tipo especificado
        if tipo_rnn == 'SimpleRNN':
            model.add(SimpleRNN(hidden_units, return_sequences=True, 
                            dropout=dropout, recurrent_dropout=recurrent_dropout))
        elif tipo_rnn == 'LSTM':
            model.add(LSTM(hidden_units, return_sequences=True, 
                        dropout=dropout, recurrent_dropout=recurrent_dropout))
        elif tipo_rnn == 'GRU':
            model.add(GRU(hidden_units, return_sequences=True, 
                        dropout=dropout, recurrent_dropout=recurrent_dropout))
        else:
            raise ValueError(f"Tipo de RNN no soportado: {tipo_rnn}")
        
        # Capa de salida
        model.add(Dense(vocab_size, activation='softmax'))
        
        # Compilación del modelo
        model.compile(loss='sparse_categorical_crossentropy', optimizer='rmsprop')
    
    return model

#### Modelo 1: SimpleRNN

In [ ]:
# Crear modelo SimpleRNN
model_simplernn = crear_modelo(tipo_rnn='SimpleRNN', hidden_units=200)
model_simplernn.summary()

/home/gury/python/pln/lib/python3.12/site-packages/keras/src/layers/core/wrapper.py:27: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ time_distributed                │ (None, None, 62)       │             0 │
│ (TimeDistributed)               │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ simple_rnn (SimpleRNN)          │ (None, None, 200)      │        52,600 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, None, 62)       │        12,462 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 65,062 (254.15 KB)

 Trainable params: 65,062 (254.15 KB)

 Non-trainable params: 0 (0.00 B)

In [ ]:
# Entrenamiento del modelo SimpleRNN
# Usar batch_size más pequeño para evitar problemas de memoria
history_ppl_simplernn = []
hist_simplernn = model_simplernn.fit(X, y, epochs=20, 
                                      callbacks=[PplCallback(tokenized_sentences_val, history_ppl_simplernn, patience=5, batch_size=32)], 
                                      batch_size=64, verbose=1)

Tamaño del dataset: 940067 secuencias
Memoria aproximada: 0.70 GB

✓ GPU disponible y funcional para entrenamiento
Iniciando entrenamiento con batch_size=64
Epoch 1/20
Epoch 1/20


I0000 00:00:1765118778.417285  143639 service.cc:145] XLA service 0x7dab50006a70 initialized for platform CUDA (this does not guarantee that XLA will be used). Devices:
I0000 00:00:1765118778.417350  143639 service.cc:153]   StreamExecutor device (0): NVIDIA GeForce GTX 1050, Compute Capability 6.1
2025-12-07 11:46:18.803886: I tensorflow/compiler/mlir/tensorflow/utils/dump_mlir_util.cc:268] disabling MLIR crash reproducer, set env var `MLIR_CRASH_REPRODUCER_DIRECTORY` to enable.
W0000 00:00:1765118778.838716  143639 assert_op.cc:38] Ignoring Assert operator compile_loss/sparse_categorical_crossentropy/SparseSoftmaxCrossEntropyWithLogits/assert_equal_1/Assert/Assert
2025-12-07 11:46:18.926066: I external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:465] Loaded cuDNN version 8907
2025-12-07 11:46:18.803886: I tensorflow/compiler/mlir/tensorflow/utils/dump_mlir_util.cc:268] disabling MLIR crash reproducer, set env var `MLIR_CRASH_REPRODUCER_DIRECTORY` to enable.
W0000 00:00:1765118778.

   13/14689 ━━━━━━━━━━━━━━━━━━━━ 3:23 14ms/step - loss: 3.6619

I0000 00:00:1765118783.116371  143639 device_compiler.h:188] Compiled cluster using XLA!  This line is logged at most once for the lifetime of the process.


14686/14689 ━━━━━━━━━━━━━━━━━━━━ 0s 14ms/step - loss: 1.9715

W0000 00:00:1765118987.742688  143635 assert_op.cc:38] Ignoring Assert operator compile_loss/sparse_categorical_crossentropy/SparseSoftmaxCrossEntropyWithLogits/assert_equal_1/Assert/Assert


14689/14689 ━━━━━━━━━━━━━━━━━━━━ 0s 14ms/step - loss: 1.9714

#### Modelo 2: LSTM

In [ ]:
# Crear modelo LSTM
model_lstm = crear_modelo(tipo_rnn='LSTM', hidden_units=200)
model_lstm.summary()

In [ ]:
# Entrenamiento del modelo LSTM
history_ppl_lstm = []
hist_lstm = model_lstm.fit(X, y, epochs=20, 
                            callbacks=[PplCallback(tokenized_sentences_val, history_ppl_lstm, patience=5, batch_size=32)], 
                            batch_size=64, verbose=1)

#### Modelo 3: GRU

In [ ]:
# Crear modelo GRU
model_gru = crear_modelo(tipo_rnn='GRU', hidden_units=200)
model_gru.summary()

In [ ]:
# Entrenamiento del modelo GRU
history_ppl_gru = []
hist_gru = model_gru.fit(X, y, epochs=20, 
                         callbacks=[PplCallback(tokenized_sentences_val, history_ppl_gru, patience=5, batch_size=32)], 
                         batch_size=64, verbose=1)

### Comparación de modelos

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns

# Visualización de la perplejidad durante el entrenamiento
fig, axes = plt.subplots(1, 3, figsize=(18, 5))

# SimpleRNN
if len(history_ppl_simplernn) > 0:
    epoch_count = range(1, len(history_ppl_simplernn) + 1)
    axes[0].plot(epoch_count, history_ppl_simplernn, marker='o')
    axes[0].set_title('SimpleRNN - Perplejidad en Validación')
    axes[0].set_xlabel('Época')
    axes[0].set_ylabel('Perplejidad')
    axes[0].grid(True)

# LSTM
if len(history_ppl_lstm) > 0:
    epoch_count = range(1, len(history_ppl_lstm) + 1)
    axes[1].plot(epoch_count, history_ppl_lstm, marker='o', color='orange')
    axes[1].set_title('LSTM - Perplejidad en Validación')
    axes[1].set_xlabel('Época')
    axes[1].set_ylabel('Perplejidad')
    axes[1].grid(True)

# GRU
if len(history_ppl_gru) > 0:
    epoch_count = range(1, len(history_ppl_gru) + 1)
    axes[2].plot(epoch_count, history_ppl_gru, marker='o', color='green')
    axes[2].set_title('GRU - Perplejidad en Validación')
    axes[2].set_xlabel('Época')
    axes[2].set_ylabel('Perplejidad')
    axes[2].grid(True)

plt.tight_layout()
plt.show()

# Comparación de perplejidad mínima
print("\n=== Resumen de Resultados ===")
if len(history_ppl_simplernn) > 0:
    print(f"SimpleRNN - Perplejidad mínima: {min(history_ppl_simplernn):.4f}")
if len(history_ppl_lstm) > 0:
    print(f"LSTM - Perplejidad mínima: {min(history_ppl_lstm):.4f}")
if len(history_ppl_gru) > 0:
    print(f"GRU - Perplejidad mínima: {min(history_ppl_gru):.4f}")

In [ ]:
# Cargar el mejor modelo guardado para hacer inferencia
best_model = keras.models.load_model('best_model.keras')
print("Mejor modelo cargado correctamente")

## Generación de secuencias

### Funciones auxiliares de codificación y decodificación

In [ ]:
# Funcionalidades para hacer encoding y decoding

def encode(text, max_length=max_context_size):
    """Codifica texto en secuencia de índices"""
    encoded = [char2idx[ch] for ch in text.lower()]
    encoded = pad_sequences([encoded], maxlen=max_length, padding='pre')
    return encoded

def decode(seq):
    """Decodifica secuencia de índices a texto"""
    return ''.join([idx2char[ch] for ch in seq])

### 1. Greedy Search

La estrategia más simple: en cada paso seleccionar el carácter con mayor probabilidad.

In [ ]:
def generate_seq_greedy(model, seed_text, max_length, n_chars):
    """
    Generación de secuencias con greedy search.
    
    Args:
        model: modelo entrenado
        seed_text: texto de entrada (contexto inicial)
        max_length: máxima longitud de la secuencia de entrada
        n_chars: números de caracteres a generar
        
    Returns:
        output_text: texto generado
    """
    output_text = seed_text
    
    # Generar un número fijo de caracteres
    for _ in range(n_chars):
        # Codificamos
        encoded = [char2idx[ch] for ch in output_text.lower()]
        # Padding si es necesario
        encoded = pad_sequences([encoded], maxlen=max_length, padding='pre')
        
        # Predicción - seleccionamos el carácter con mayor probabilidad
        y_hat = np.argmax(model.predict(encoded, verbose=0)[0, -1, :])
        
        # Decodificamos y agregamos al texto
        out_char = idx2char[y_hat]
        output_text += out_char
        
    return output_text

In [ ]:
# Prueba de greedy search con diferentes contextos
input_texts = [
    'una tarde',
    'raskolnikoff',
    'san petersburgo',
    'la vieja'
]

print("=== GENERACIÓN CON GREEDY SEARCH ===\n")
for input_text in input_texts:
    generated = generate_seq_greedy(best_model, input_text, max_length=max_context_size, n_chars=100)
    print(f"Contexto: '{input_text}'")
    print(f"Generado: {generated}\n")
    print("-" * 80 + "\n")

### 2. Beam Search

Implementación de beam search determinístico y estocástico con temperatura.

In [ ]:
from scipy.special import softmax

def select_candidates(pred, num_beams, vocab_size, history_probs, history_tokens, temp, mode):
    """
    Selecciona candidatos para el beam search.
    
    Args:
        pred: predicciones del modelo
        num_beams: número de beams a mantener
        vocab_size: tamaño del vocabulario
        history_probs: historial de probabilidades logarítmicas
        history_tokens: historial de tokens
        temp: temperatura para muestreo estocástico
        mode: 'det' para determinístico, 'sto' para estocástico
        
    Returns:
        nuevas probabilidades e historial de tokens
    """
    # Colectar todas las probabilidades para la siguiente búsqueda
    pred_large = []
    
    for idx, pp in enumerate(pred):
        pred_large.extend(np.log(pp + 1E-10) + history_probs[idx])
    
    pred_large = np.array(pred_large)
    
    # Criterio de selección
    if mode == 'det':
        # Beam search determinístico
        idx_select = np.argsort(pred_large)[::-1][:num_beams]
    elif mode == 'sto':
        # Beam search con muestreo aleatorio
        idx_select = np.random.choice(np.arange(pred_large.shape[0]), 
                                     num_beams, 
                                     p=softmax(pred_large / temp))
    else:
        raise ValueError(f'Modo de selección incorrecto. {mode} fue dado. "det" y "sto" son soportados.')
    
    # Traducir a índices de token en el vocabulario
    new_history_tokens = np.concatenate((np.array(history_tokens)[idx_select // vocab_size],
                                        np.array([idx_select % vocab_size]).T),
                                       axis=1)
    
    # Devolver el producto de las probabilidades (log) y la secuencia de tokens seleccionados
    return pred_large[idx_select.astype(int)], new_history_tokens.astype(int)


def beam_search(model, num_beams, num_chars, input_text, temp=1, mode='det'):
    """
    Implementa beam search para generación de texto.
    
    Args:
        model: modelo entrenado
        num_beams: número de beams a mantener
        num_chars: número de caracteres a generar
        input_text: texto de contexto inicial
        temp: temperatura (solo para modo estocástico)
        mode: 'det' para determinístico, 'sto' para estocástico
        
    Returns:
        secuencias generadas
    """
    # Primera iteración
    
    # Codificar
    encoded = encode(input_text)
    
    # Primera predicción
    y_hat = model.predict(encoded, verbose=0)[0, -1, :]
    
    # Obtener tamaño del vocabulario
    vocab_size_local = y_hat.shape[0]
    
    # Inicializar historial
    history_probs = [0] * num_beams
    history_tokens = [encoded[0]] * num_beams
    
    # Seleccionar num_beams candidatos
    history_probs, history_tokens = select_candidates([y_hat],
                                                      num_beams,
                                                      vocab_size_local,
                                                      history_probs,
                                                      history_tokens,
                                                      temp,
                                                      mode)
    
    # Loop de beam search
    for i in range(num_chars - 1):
        preds = []
        
        for hist in history_tokens:
            # Actualizar secuencia de tokens
            input_update = np.array([hist[i + 1:]]).copy()
            
            # Predicción
            y_hat = model.predict(input_update, verbose=0)[0, -1, :]
            
            preds.append(y_hat)
        
        history_probs, history_tokens = select_candidates(preds,
                                                          num_beams,
                                                          vocab_size_local,
                                                          history_probs,
                                                          history_tokens,
                                                          temp,
                                                          mode)
    
    return history_tokens[:, -(len(input_text) + num_chars):]

### 3. Beam Search Determinístico

Prueba de beam search determinístico con diferentes números de beams.

In [ ]:
# Función para crear diferentes arquitecturas de modelos (OPTIMIZADA PARA GPU)
def crear_modelo(tipo_rnn='SimpleRNN', hidden_units=200, dropout=0.1, recurrent_dropout=0.1, embedding_dim=64):
    """
    Crea un modelo de lenguaje con la arquitectura especificada.
    Optimizado para GPU usando Embedding en lugar de One-Hot Encoding.
    
    Args:
        tipo_rnn: tipo de capa recurrente ('SimpleRNN', 'LSTM', 'GRU')
        hidden_units: número de unidades ocultas en la capa recurrente
        dropout: tasa de dropout
        recurrent_dropout: tasa de dropout recurrente
        embedding_dim: dimensión del embedding (más eficiente que OHE)
    
    Returns:
        modelo compilado
    """
    model = Sequential()
    
    # Usar Embedding en lugar de One-Hot Encoding (mucho más eficiente en GPU)
    model.add(Embedding(input_dim=vocab_size, output_dim=embedding_dim, 
                        input_length=max_context_size))
    
    # Capa recurrente según el tipo especificado
    if tipo_rnn == 'SimpleRNN':
        model.add(SimpleRNN(hidden_units, return_sequences=True, 
                           dropout=dropout, recurrent_dropout=recurrent_dropout))
    elif tipo_rnn == 'LSTM':
        model.add(LSTM(hidden_units, return_sequences=True, 
                      dropout=dropout, recurrent_dropout=recurrent_dropout))
    elif tipo_rnn == 'GRU':
        model.add(GRU(hidden_units, return_sequences=True, 
                     dropout=dropout, recurrent_dropout=recurrent_dropout))
    else:
        raise ValueError(f"Tipo de RNN no soportado: {tipo_rnn}")
    
    # Capa de salida
    model.add(Dense(vocab_size, activation='softmax'))
    
    # Compilación del modelo con optimizador Adam (más estable en GPU)
    model.compile(loss='sparse_categorical_crossentropy', 
                 optimizer='adam',
                 metrics=['accuracy'])
    
    return model

### 4. Beam Search Estocástico con Temperatura

Prueba de beam search estocástico explorando el efecto de la temperatura. 
- **Temperatura baja** (< 1): hace la distribución más "determinística", favoreciendo tokens con alta probabilidad
- **Temperatura alta** (> 1): hace la distribución más uniforme, favoreciendo diversidad y exploración
- **Temperatura = 1**: distribución original sin modificar

In [ ]:
# Prueba de beam search estocástico con diferentes temperaturas
input_text = "raskolnikoff"
num_beams = 5
num_chars_to_generate = 100

print("=== BEAM SEARCH ESTOCÁSTICO - EFECTO DE LA TEMPERATURA ===\n")
print(f"Contexto inicial: '{input_text}'")
print(f"Número de beams: {num_beams}")
print(f"Caracteres a generar: {num_chars_to_generate}\n")

temperaturas = [0.3, 0.7, 1.0, 1.5, 2.0]

for temp in temperaturas:
    print(f"\n{'='*80}")
    print(f"TEMPERATURA = {temp}")
    print('='*80)
    
    salidas = beam_search(best_model, num_beams=num_beams, num_chars=num_chars_to_generate, 
                         input_text=input_text, temp=temp, mode='sto')
    
    for idx, salida in enumerate(salidas[:3], 1):  # Mostrar las top 3
        texto_generado = decode(salida)
        print(f"{idx}. {texto_generado}")
    print()

# Entrenamiento del modelo SimpleRNN
# IMPORTANTE: Reducir dataset si hay problemas de memoria
print(f"Tamaño del dataset completo: {len(X)} secuencias")
print(f"Memoria aproximada: {X.nbytes / (1024**3):.2f} GB")

# Usar un subconjunto del dataset si es necesario (descomentar si sigue habiendo problemas)
# X_train = X[:100000]  # Usar solo 100k secuencias
# y_train = y[:100000]

# Entrenamiento con batch pequeño para GTX 1050
history_ppl_simplernn = []

with tf.device('/GPU:0'):
    hist_simplernn = model_simplernn.fit(
        X, y, 
        epochs=20, 
        callbacks=[PplCallback(tokenized_sentences_val, history_ppl_simplernn, patience=5, batch_size=32)], 
        batch_size=64,  # Batch pequeño para GPU de 4GB
        verbose=1
    )

In [ ]:
# Comparación de estrategias con el mismo contexto
input_text = "san petersburgo"
n_chars = 150

print("="*100)
print("COMPARACIÓN DE ESTRATEGIAS DE GENERACIÓN")
print("="*100)
print(f"\nContexto inicial: '{input_text}'")
print(f"Caracteres a generar: {n_chars}\n")

print("\n" + "="*100)
print("1. GREEDY SEARCH")
print("="*100)
resultado_greedy = generate_seq_greedy(best_model, input_text, max_context_size, n_chars)
print(resultado_greedy)

print("\n" + "="*100)
print("2. BEAM SEARCH DETERMINÍSTICO (10 beams)")
print("="*100)
salidas_det = beam_search(best_model, num_beams=10, num_chars=n_chars, 
                         input_text=input_text, mode='det')
print(decode(salidas_det[0]))

print("\n" + "="*100)
print("3. BEAM SEARCH ESTOCÁSTICO (10 beams, temperatura=0.7)")
print("="*100)
salidas_sto_07 = beam_search(best_model, num_beams=10, num_chars=n_chars, 
                             input_text=input_text, temp=0.7, mode='sto')
print(decode(salidas_sto_07[0]))

print("\n" + "="*100)
print("4. BEAM SEARCH ESTOCÁSTICO (10 beams, temperatura=1.5)")
print("="*100)
salidas_sto_15 = beam_search(best_model, num_beams=10, num_chars=n_chars, 
                             input_text=input_text, temp=1.5, mode='sto')
print(decode(salidas_sto_15[0]))
print("\n")

In [ ]:
# Crear modelo LSTM
model_lstm = crear_modelo(tipo_rnn='LSTM', hidden_units=200, embedding_dim=64)
model_lstm.summary()

## Conclusiones

### Sobre el corpus y pre-procesamiento
- Se utilizó el texto completo de "Crimen y Castigo" de Dostoyevski en español, con aproximadamente 500,000 caracteres.
- El corpus fue tokenizado a nivel de caracteres, creando un vocabulario de aproximadamente 60-80 caracteres únicos.
- Se utilizó una ventana de contexto de 100 caracteres para capturar dependencias de largo alcance.
- Los datos se dividieron en 90% entrenamiento y 10% validación.

### Sobre las arquitecturas
Se probaron tres arquitecturas de redes recurrentes:
1. **SimpleRNN**: Arquitectura básica, más simple pero con limitaciones para capturar dependencias largas.
2. **LSTM**: Maneja mejor las dependencias de largo plazo gracias a sus puertas de olvido y memoria.
3. **GRU**: Similar a LSTM pero con menos parámetros, buen balance entre eficiencia y capacidad.

La perplejidad en validación permitió seleccionar el mejor modelo mediante early stopping.

### Sobre las estrategias de generación

1. **Greedy Search**:
   - Estrategia más simple: selecciona siempre el carácter con mayor probabilidad.
   - Genera texto coherente pero puede caer en repeticiones.
   - Computacionalmente eficiente.

2. **Beam Search Determinístico**:
   - Mantiene múltiples hipótesis (beams) simultáneamente.
   - Explora más el espacio de búsqueda que greedy search.
   - Mayor número de beams → mayor calidad pero mayor costo computacional.

3. **Beam Search Estocástico con Temperatura**:
   - **Temperatura baja (0.3-0.7)**: Texto más conservador y coherente, favorece tokens probables.
   - **Temperatura media (1.0)**: Balance entre coherencia y diversidad.
   - **Temperatura alta (1.5-2.0)**: Mayor creatividad y diversidad, pero potencialmente menos coherente.

### Observaciones finales
- El modelo de lenguaje por caracteres permite generar texto fluido en español.
- La temperatura es un hiperparámetro crucial para controlar el balance creatividad/coherencia.
- Las arquitecturas LSTM y GRU generalmente superan a SimpleRNN en la captura de contexto.
- El corpus literario permite generar texto con estilo similar a la novela original.

In [ ]:
# Entrenamiento del modelo GRU
history_ppl_gru = []

with tf.device('/GPU:0'):
    hist_gru = model_gru.fit(
        X, y, 
        epochs=20, 
        callbacks=[PplCallback(tokenized_sentences_val, history_ppl_gru, patience=5, batch_size=32)], 
        batch_size=64,
        verbose=1
    )

In [ ]:
# Crear modelo GRU
model_gru = crear_modelo(tipo_rnn='GRU', hidden_units=200, embedding_dim=64)
model_gru.summary()

#### Modelo 3: GRU

In [ ]:
# Entrenamiento del modelo LSTM
history_ppl_lstm = []

with tf.device('/GPU:0'):
    hist_lstm = model_lstm.fit(
        X, y, 
        epochs=20, 
        callbacks=[PplCallback(tokenized_sentences_val, history_ppl_lstm, patience=5, batch_size=32)], 
        batch_size=64,
        verbose=1
    )